In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# get rid of GPU logging messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"]='3,4,7'

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers

import tensorflow_addons as tfa

/scratch/student/cb2078/env/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/scratch/student/cb2078/env/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: 

In [3]:
# if using a gpu limit its memory usage
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print('e',e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 3 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 3 Logical GPUs


In [4]:
#Load the training and testing dataset
#TODO Validation dataset
(ds_train, ds_test, ds_val), ds_info = tfds.load('oxford_flowers102',
                                            split=['train', 'test', 'validation'],
                                            with_info=True,
                                            shuffle_files=True,
                                            as_supervised=True)

#This gets the images ready
shape = (800, 800, 3)
small_shape = (256, 256, 3)
n_labels = 102

def normalize_img(img, label):
    img = tf.image.resize(img,shape[:2])
    img = tf.cast(img, tf.float32) / 255
    return (img, label)

def resize_img(img, label):
    img = tf.image.resize(img,small_shape[:2])
    return (img, label)

def random_flip_img(img, label):
    img = tf.image.random_flip_left_right(img)
    return (img, label)

def random_crop(img, label):
    img = tf.image.resize(img, (int(shape[0] * 1.25), int(shape[1] * 1.25)))
    img = tf.image.random_crop(img, shape)
    return (img, label)

def random_move_img(img, label):
    offset = shape[0] / 6
    img = tfa.image.translate(
        img,
        tf.random.uniform(shape = [2], minval = -offset, maxval = offset)
    )
    return img, label

def random_rotate(img, label):
    img = tfa.image.rotate(img, tf.random.uniform(shape = [1], minval = -0.5, maxval = 0.5))
    return img, label

def random_cutout(img, label):
    cutout_size = (shape[0] // 64) ** 2
    n_cutouts = 10
    for _ in range(n_cutouts):
        # image.cutout needs a 4d tensor but img is a 3d tensor
        img = tfa.image.cutout([img], cutout_size, tf.random.uniform(
            shape = [2], maxval = shape[0],
            dtype = tf.dtypes.int32
        ))
        img = img[0,]
    return img, label

def random_image_warp(img, label):
    n_warp_points = 25
    offset = shape[0] // 32 
    offsets = tf.random.uniform(shape = [n_warp_points, 2], minval = -offset, maxval = offset)
    src_points = tf.random.uniform(shape = [n_warp_points, 2], maxval = shape[0])
    dest_points = src_points + offsets
    img, flow = tfa.image.sparse_image_warp([img], [src_points], [dest_points])
    img = img[0,]
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)

functions = [
    random_crop,
    random_move_img,
    random_rotate,
    random_cutout,
    random_image_warp,
    random_flip_img
]
repetitions = 7
ds_train_orig = ds_train
for function in functions:
    for _ in range(repetitions):
        ds_augmented = ds_train_orig.map(function, num_parallel_calls=AUTOTUNE)
        ds_train = ds_train.concatenate(ds_augmented)
#     print('-' * 20, function.__name__, '-'*20)
#     tfds.visualization.show_examples(ds_augmented, ds_info)
ds_train = ds_train.map(resize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)
        
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.map(resize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

ds_val = ds_val.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.map(resize_img, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.cache()
ds_val = ds_val.shuffle(1000)
ds_val = ds_val.batch(BATCH_SIZE)
ds_val = ds_val.prefetch(AUTOTUNE)

print(ds_train.cardinality().numpy() * BATCH_SIZE)

43904
43904


In [5]:
#Making the NN model
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu",input_shape=shape),
#     tf.keras.layers.BatchNormalization(),
    
#     tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=2),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256, activation="relu"),
#     tf.keras.layers.Dense(n_labels, activation="softmax")]
# )

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():   
    #Making the NN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(128, (3,3), padding='same', activation="relu",input_shape=small_shape),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        #tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv2D(256, (3,3), padding='same', activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Dropout(0.8),

        tf.keras.layers.Conv2D(256, (3,3), padding='same', activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Dropout(0.8),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(n_labels, activation="softmax")]
    )

    #Learning rate scheduler (EXPLAIN THIS)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=10000,
        decay_rate=0.9
    )


    model.compile(
        optimizer=keras.optimizers.Adam(lr_schedule),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


Model: "sequential"
Model: "sequential"
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 128)     3584      
 conv2d (Conv2D)             (None, 256, 256, 128)     3584      
                                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 128)    0         
 max_pooling2d (MaxPooling2D  (None, 128, 128, 128)    0         
 )                                                               
 )                                                               
                                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 

In [6]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
#Train the model 
history = model.fit(
    ds_train,
    epochs=100,
    validation_data=ds_val)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(ds_test, verbose=2)

Epoch 1/100
Epoch 1/100


/scratch/student/cb2078/env/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/scratch/student/cb2078/env/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


192/686 [=======>......................] - ETA: 4:19 - loss: 4.7213 - accuracy: 0.0084